<a href="https://colab.research.google.com/github/EugeneHsiung/datasci_6_anova/blob/main/datasci_6_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing packages**

In [1]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

In [5]:
# fetch dataset
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)

/usr/local/lib/python3.10/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [6]:
# data (as pandas dataframes)
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets

In [8]:
df = pd.DataFrame(X)
df

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),NaN,6,25,1,1,NaN,Pediatrics-Endocrinology,...,No,No,No,No,No,No,No,No,No,No
1,Caucasian,Female,[10-20),NaN,1,1,7,3,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
2,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,NaN,NaN,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),NaN,1,1,7,2,NaN,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),NaN,1,1,7,1,NaN,NaN,...,No,No,Steady,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101762,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,MC,NaN,...,No,No,Steady,No,No,No,No,No,No,Yes
101763,Caucasian,Male,[70-80),NaN,1,1,7,1,MC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
101764,Caucasian,Female,[80-90),NaN,2,3,7,10,MC,Surgery-General,...,No,No,Up,No,No,No,No,No,Ch,Yes


In [9]:
df.sample(10)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
7893,Caucasian,Female,[70-80),NaN,5,6,17,7,NaN,Gastroenterology,...,No,No,Steady,No,No,No,No,No,No,Yes
26080,AfricanAmerican,Female,[50-60),NaN,2,1,1,2,UN,Surgery-Neuro,...,No,No,Steady,No,No,No,No,No,Ch,Yes
43047,Caucasian,Female,[70-80),[75-100),2,1,5,3,MC,NaN,...,No,No,No,No,No,No,No,No,Ch,Yes
18029,Caucasian,Female,[70-80),NaN,2,18,2,10,NaN,NaN,...,No,No,No,No,No,No,No,No,Ch,Yes
57719,Caucasian,Male,[70-80),NaN,1,3,7,6,HM,NaN,...,No,No,No,No,No,No,No,No,No,Yes
44433,Caucasian,Female,[30-40),NaN,1,1,7,1,BC,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
92921,Caucasian,Male,[90-100),NaN,2,6,7,10,MC,Emergency/Trauma,...,No,No,Up,No,No,No,No,No,Ch,Yes
28751,Hispanic,Female,[50-60),NaN,6,1,7,1,NaN,Cardiology,...,No,No,No,No,No,No,No,No,No,No
66061,Caucasian,Female,[80-90),NaN,5,3,1,3,MC,NaN,...,No,No,Up,No,No,No,No,No,Ch,Yes
87598,Caucasian,Female,[40-50),NaN,1,1,7,6,PO,NaN,...,No,No,Down,No,No,No,No,No,Ch,Yes
